<h1>Chapter 7 - Advanced Text Generation Techniques and Tools</h1>
<i>Going beyond prompt engineering.</i>

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter07/Chapter%207%20-%20Advanced%20Text%20Generation%20Techniques%20and%20Tools.ipynb)

---

This notebook is for Chapter 7 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>

### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [1]:
%pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
%pip install llama-cpp-python==0.2.69

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Loading an LLM

In [ ]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

In [3]:
%pip install llama-cpp-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.llms import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [5]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

''

### Chains

In [6]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [7]:
basic_chain = prompt | llm

In [8]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

' Hello Maarten, the answer to 1 + 1 is 2.'

### Multiple Chains

In [9]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

/tmp/ipykernel_38454/3105618174.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [10]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother',
 'title': ' "Whispers of Love: A Journey Through Grief"'}

In [11]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [12]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [13]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [14]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "Finding Warmth in Grief: A Tale of Lily\'s Journey"',
 'character': " Lily is an empathetic and resilient young girl who, after losing her beloved mother to illness, embarks on a transformative journey to find solace and healing amidst the depths of grief. Her kind heart and unyielding spirit lead her to unexpected friendships and self-discoveries that help her navigate through life's challenges with grace and newfound strength.\n\nLily, an introverted yet determined girl, is forced to grow beyond her comfort zone as she seeks out ways to cope with the loss of her mother; this quest unveils both heartbreaking vulnerabilities and inspiring moments of courage that gradually shape her into a pillar of support for others in their own grief.",
 'story': " Finding Warmth in Grief: A Tale of Lily's Journey tells the poignant tale of an empathetic and resilient young girl named Lily, who after experiencing the tragic loss of her beloved m

# Memory

In [15]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

' Hello Maarten, the answer to 1 + 1 is 2.'

In [16]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have access to personal data of individuals. If you need help with something specific, feel free to ask!"

## ConversationBuffer

In [17]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [18]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipykernel_38454/1531358956.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [19]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Maarten. What is 1 + 1?',
 'chat_history': '',
 'text': " Hello Maarten! The answer to what 1 + 1 equals is 2. It's a basic arithmetic operation in mathematics where you add the two numbers together."}

In [20]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten. What is 1 + 1?\nAI:  Hello Maarten! The answer to what 1 + 1 equals is 2. It's a basic arithmetic operation in mathematics where you add the two numbers together.",
 'text': " Your name is mentioned as Maarten by the human in this conversation.\n\nAs for your identity, I'm an AI digital assistant and don't have a personal name, but you can refer to me as the AI."}

## ConversationBufferMemoryWindow

In [21]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipykernel_38454/535028085.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [22]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, my name isn't programmed to remember personal details, but I can certainly help with the math question! 1 + 1 equals 2.",
 'text': ' 3 + 3 equals 6.'}

In [23]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, my name isn't programmed to remember personal details, but I can certainly help with the math question! 1 + 1 equals 2.\nHuman: What is 3 + 3?\nAI:  3 + 3 equals 6.",
 'text': ' Your name, as mentioned in the conversation, is Maarten.\n\nAnd to answer your math question again, 3 + 3 equals 6.'}

In [24]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI:  3 + 3 equals 6.\nHuman: What is my name?\nAI:  Your name, as mentioned in the conversation, is Maarten.\n\nAnd to answer your math question again, 3 + 3 equals 6.',
 'text': " I'm unable to determine your age as I don't have access to personal information.\nAnswer: I cannot answer that question."}

## ConversationSummary

In [25]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [26]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

/tmp/ipykernel_38454/4216441337.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [27]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Maarten introduces himself and asks a simple math question about the sum of 1 + 1, to which the AI confirms that it equals 2, providing an explanation on addition as a basic arithmetic operation.',
 'text': ' Your name was not mentioned in our previous conversation. You introduced yourself as Maarten. How may I assist you further with math or any other queries?'}

In [28]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Maarten introduces himself and inquires about the sum of 1 + 1, to which the AI confirms it equals 2, explaining addition as a basic arithmetic operation. The AI also reminds Maarten that his name was not provided earlier but recognizes him from the introduction. It stands ready to assist with any math problems or additional queries he may have.',
 'text': ' The first question you asked was, "what is 1+1?"'}

In [29]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': ' Maarten introduces himself and asks about the sum of 1 + 1, to which the AI confirms it equals 2. The AI also reminds Maarten that his name wasn\'t mentioned in their previous interaction but recognizes him from this introduction. It stands ready to assist with any math problems or additional queries he may have. Additionally, when asked about the first question he posed, the human inquires and the AI confirms it was "what is 1+1?". The AI explains addition as a fundamental arithmetic operation before offering further assistance.'}

# Agents

In [ ]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
#os.environ["OPENAI_API_KEY"] = "your_api_key_here"
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [37]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [38]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools|
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [39]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [40]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
I should use a web search engine to find the current price of a MacBook Pro in USD and then use a calculator to convert it to EUR.
Action: duckduck
Action Input: "current price of MacBook Pro in USD"snippet: Apple's 2024 MacBook Pro 16-inch can be equipped with an M4 Pro or M4 Max chip. Retail prices start at $2,499 USD , but every configuration is on sale, with the best prices in this guide knocking hundred of dollars off the laptop of your choosing. You can also browse a selection of the top 16-inch MacBook Pro deals in our dedicated roundup., title: MacBook Pro 16-inch 2024 M4 Best Sale Price Deals, link: https://prices.appleinsider.com/macbook-pro-16-inch-m4, snippet: Apple is refreshing the 14-inch MacBook Pro with its new M5 chip, which promises to further enhance graphics and AI workload performance. The updated MacBook is for students, business users, and ..., title: Apple's New 14-Inch MacBook Pro Arrives With M5 Chip, $1,599 Starting Pr

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD is $2,499. It would cost approximately €2,124.15 in EUR with an exchange rate of 0.85 EUR for 1 USD.'}